In [15]:
from musetalk.utils.utils import load_all_model


vae, unet, pe = load_all_model(
    unet_model_path="./models/musetalk/pytorch_model.bin",
    vae_type="sd-vae",
    unet_config="./models/musetalk/musetalk.json",
    device='cuda'
)

An error occurred while trying to fetch models/sd-vae: Error no file named diffusion_pytorch_model.safetensors found in directory models/sd-vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


load unet model from ./models/musetalk/pytorch_model.bin


In [17]:
unet.model

UNet2DConditionModel(
  (conv_in): Conv2d(8, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features=320, out_fe

In [3]:
vae.vae

AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

In [5]:
unet.model

UNet2DConditionModel(
  (conv_in): Conv2d(8, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features=320, out_fe

In [10]:
pe

PositionalEncoding()

In [3]:
import math
import os

import librosa
import numpy as np
import torch
from einops import rearrange
from transformers import AutoFeatureExtractor


class AudioProcessor:
    def __init__(self, feature_extractor_path="openai/whisper-tiny"):
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(feature_extractor_path)

    def get_audio_feature(self, wav_path, start_index=0, weight_dtype=None):
        if not os.path.exists(wav_path):
            return None
        librosa_output, sampling_rate = librosa.load(wav_path, sr=16000)
        assert sampling_rate == 16000
        # Split audio into 30s segments
        segment_length = 30 * sampling_rate
        segments = [librosa_output[i:i + segment_length] for i in range(0, len(librosa_output), segment_length)]

        features = []
        for segment in segments:
            audio_feature = self.feature_extractor(
                segment,
                return_tensors="pt",
                sampling_rate=sampling_rate
            ).input_features
            if weight_dtype is not None:
                audio_feature = audio_feature.to(dtype=weight_dtype)
            features.append(audio_feature)

        return features, len(librosa_output)

    def get_whisper_chunk(
        self,
        whisper_input_features,
        device,
        weight_dtype,
        whisper,
        librosa_length,
        fps=25,
        audio_padding_length_left=2,
        audio_padding_length_right=2,
    ):
        audio_feature_length_per_frame = 2 * (audio_padding_length_left + audio_padding_length_right + 1)
        whisper_feature = []
        # Process multiple 30s mel input features
        for input_feature in whisper_input_features:
            input_feature = input_feature.to(device).to(weight_dtype)
            audio_feats = whisper.encoder(input_feature, output_hidden_states=True).hidden_states
            audio_feats = torch.stack(audio_feats, dim=2)
            whisper_feature.append(audio_feats)

        whisper_feature = torch.cat(whisper_feature, dim=1)
        # Trim the last segment to remove padding
        sr = 16000
        audio_fps = 50
        fps = int(fps)
        whisper_idx_multiplier = audio_fps / fps
        num_frames = math.floor((librosa_length / sr) * fps)
        actual_length = math.floor((librosa_length / sr) * audio_fps)
        whisper_feature = whisper_feature[:,:actual_length,...]

        # Calculate padding amount
        padding_nums = math.ceil(whisper_idx_multiplier)
        # Add padding at start and end
        whisper_feature = torch.cat([
            torch.zeros_like(whisper_feature[:, :padding_nums * audio_padding_length_left]),
            whisper_feature,
            # Add extra padding to prevent out of bounds
            torch.zeros_like(whisper_feature[:, :padding_nums * 3 * audio_padding_length_right])
        ], 1)

        audio_prompts = []
        for frame_index in range(num_frames):
            try:
                audio_index = math.floor(frame_index * whisper_idx_multiplier)
                audio_clip = whisper_feature[:, audio_index: audio_index + audio_feature_length_per_frame]
                assert audio_clip.shape[1] == audio_feature_length_per_frame
                audio_prompts.append(audio_clip)
            except Exception as e:
                print(f"Error occurred: {e}")
                print(f"whisper_feature.shape: {whisper_feature.shape}")
                print(f"audio_clip.shape: {audio_clip.shape}")
                print(f"num frames: {num_frames}, fps: {fps}, whisper_idx_multiplier: {whisper_idx_multiplier}")
                print(f"frame_index: {frame_index}, audio_index: {audio_index}-{audio_index + audio_feature_length_per_frame}")
                exit()

        audio_prompts = torch.cat(audio_prompts, dim=0)  # T, 10, 5, 384
        audio_prompts = rearrange(audio_prompts, 'b c h w -> b (c h) w')
        return audio_prompts




In [ ]:
audio_processor = AudioProcessor()
wav_path = './data/audio/piradoba.wav'
audio_feature, librosa_feature_length = audio_processor.get_audio_feature(wav_path)


/home/lukadarsalia/Documents/MuseTalk/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


AttributeError: 'list' object has no attribute 'shape'

In [5]:
audio_feature

[tensor([[[-0.4280, -0.0149, -0.1384,  ..., -0.7431, -0.7431, -0.7431],
          [-0.5044, -0.1854, -0.2437,  ..., -0.7431, -0.7431, -0.7431],
          [-0.4600, -0.2401, -0.3250,  ..., -0.7431, -0.7431, -0.7431],
          ...,
          [-0.7431, -0.7431, -0.7431,  ..., -0.7431, -0.7431, -0.7431],
          [-0.7431, -0.7431, -0.7431,  ..., -0.7431, -0.7431, -0.7431],
          [-0.7431, -0.7431, -0.7431,  ..., -0.7431, -0.7431, -0.7431]]])]

In [8]:
print("Audio Feature shape:", audio_feature[0].shape)
print("librosa_feature_length:", librosa_feature_length)

Audio Feature shape: torch.Size([1, 80, 3000])
librosa_feature_length: 106496


In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn
from typing import Optional, Tuple, Union, List
from musetalk.models.vae import VAE
from musetalk.models.unet import UNet,PositionalEncoding

from einops import rearrange


class MuseTalkGenerator(nn.Module):
    """Unified generator that wraps positional encoding, UNet, and VAE decoding."""

    def __init__(
        self,
        *,
        whisper_encoder: nn.Module,
        vae: nn.Module,
        unet: nn.Module,
        pe: nn.Module,
        audio_padding_length_left: int = 2,
        audio_padding_length_right: int = 2,
        video_fps: int = 25,
    ) -> None:
        super().__init__()
        self.whisper_encoder = whisper_encoder
        self.whisper_encoder_device = next(self.whisper_encoder.parameters()).device

        self.vae = vae
        self.vae_device = next(self.vae.parameters()).device
        self.scaling_factor = self.vae.config.scaling_factor
        self.unet = unet
        self.unet_device = next(self.unet.parameters()).device
        self.timesteps = torch.tensor([0], device=self.unet_device)
        self.pe = pe

        self.audio_padding_length_left = audio_padding_length_left
        self.audio_padding_length_right = audio_padding_length_right
        self.video_fps = video_fps

    def decode_latents(self, latents):
        """
        Decode latent variables back into an image.
        :param latents: The latent variables to decode.
        :return: A NumPy array representing the decoded image.
        """
        latents = (1/  self.scaling_factor) * latents
        image = self.vae.decode(latents.to(self.vae.dtype)).sample
        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.detach().cpu().permute(0, 2, 3, 1).float().numpy()
        image = (image * 255).round().astype("uint8")
        image = image[...,::-1] # RGB to BGR
        return image

    def get_whisper_chunk(
        self,
        whisper_input_features,
        device,
        weight_dtype,
        librosa_length,
        fps=25,
    ):
        audio_feature_length_per_frame = 2 * (self.audio_padding_length_left + self.audio_padding_length_right + 1)
        whisper_feature = []
        # Process multiple 30s mel input features
        for input_feature in whisper_input_features:
            input_feature = input_feature.to(device).to(weight_dtype)
            audio_feats = self.whisper_encoder(input_feature, output_hidden_states=True).hidden_states
            audio_feats = torch.stack(audio_feats, dim=2)
            whisper_feature.append(audio_feats)

        whisper_feature = torch.cat(whisper_feature, dim=1)
        # Trim the last segment to remove padding
        sr = 16000
        audio_fps = 50
        fps = int(fps)
        whisper_idx_multiplier = audio_fps / fps
        num_frames = math.floor((librosa_length / sr) * fps)
        actual_length = math.floor((librosa_length / sr) * audio_fps)
        whisper_feature = whisper_feature[:,:actual_length,...]

        # Calculate padding amount
        padding_nums = math.ceil(whisper_idx_multiplier)
        # Add padding at start and end
        whisper_feature = torch.cat([
            torch.zeros_like(whisper_feature[:, :padding_nums * self.audio_padding_length_left]),
            whisper_feature,
            # Add extra padding to prevent out of bounds
            torch.zeros_like(whisper_feature[:, :padding_nums * 3 * self.audio_padding_length_right])
        ], 1)

        audio_prompts = []
        for frame_index in range(num_frames):
            audio_index = math.floor(frame_index * whisper_idx_multiplier)
            audio_clip = whisper_feature[:, audio_index: audio_index + audio_feature_length_per_frame]
            audio_prompts.append(audio_clip)

        audio_prompts = torch.cat(audio_prompts, dim=0)  # T, 10, 5, 384
        audio_prompts = rearrange(audio_prompts, 'b c h w -> b (c h) w')
        return audio_prompts

    def forward(
        self,
        whisper_input_features: List[torch.Tensor],
        latent_inputs: torch.Tensor,
        frame_idx: int,
        librosa_length: int,
    ):
        whisper_chunks = self.get_whisper_chunk(
            whisper_input_features=whisper_input_features,
            device=self.whisper_encoder_device,
            weight_dtype=self.whisper_encoder.dtype,
            librosa_length=librosa_length,
            fps=self.video_fps,
        )

        batch_size = latent_inputs.shape[0]
        context_whisper_chunk = whisper_chunks[frame_idx: frame_idx+batch_size]  # shape [batch_size, 50, 384]
        
        audio_feature_batch = self.pe(context_whisper_chunk.to(self.unet_device))
        latent_batch = latent_inputs.to(device=self.unet_device, dtype=self.unet.dtype)

        pred_latents = self.unet(
            latent_batch,
            self.timesteps,
            encoder_hidden_states=audio_feature_batch,
        ).sample

        pred_latents = pred_latents.to(device=self.vae_device, dtype=self.vae.dtype)
        recon = self.decode_latents(pred_latents)
        return recon


: 

In [30]:
from musetalk.utils.utils import load_all_model


vae, unet, pe = load_all_model(
    unet_model_path="./models/musetalk/pytorch_model.bin",
    vae_type="sd-vae",
    unet_config="./models/musetalk/musetalk.json",
    device='cuda'
)

An error occurred while trying to fetch models/sd-vae: Error no file named diffusion_pytorch_model.safetensors found in directory models/sd-vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


load unet model from ./models/musetalk/pytorch_model.bin


In [31]:
pe

PositionalEncoding()

In [32]:
from transformers import WhisperModel
whisper = WhisperModel.from_pretrained('./models/whisper')
whisper = whisper.to(device='cuda', dtype=unet.model.dtype).eval()
whisper.requires_grad_(False)

WhisperModel(
  (encoder): WhisperEncoder(
    (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
    (embed_positions): Embedding(1500, 384)
    (layers): ModuleList(
      (0-3): 4 x WhisperEncoderLayer(
        (self_attn): WhisperSdpaAttention(
          (k_proj): Linear(in_features=384, out_features=384, bias=False)
          (v_proj): Linear(in_features=384, out_features=384, bias=True)
          (q_proj): Linear(in_features=384, out_features=384, bias=True)
          (out_proj): Linear(in_features=384, out_features=384, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (final_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    

In [33]:
model = MuseTalkGenerator(
    whisper_encoder=whisper.encoder,
    vae=vae.vae,
    unet=unet.model,
    pe=pe,
)

In [34]:
model

MuseTalkGenerator(
  (whisper_encoder): WhisperEncoder(
    (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
    (embed_positions): Embedding(1500, 384)
    (layers): ModuleList(
      (0-3): 4 x WhisperEncoderLayer(
        (self_attn): WhisperSdpaAttention(
          (k_proj): Linear(in_features=384, out_features=384, bias=False)
          (v_proj): Linear(in_features=384, out_features=384, bias=True)
          (q_proj): Linear(in_features=384, out_features=384, bias=True)
          (out_proj): Linear(in_features=384, out_features=384, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (final_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affi

In [ ]:
model(whisper_input_features = [torch.randn(1, 80, 3000)],
latent_inputs = torch.randn(8, 8, 32, 32),
frame_idx = torch.tensor(0, dtype=torch.long),
librosa_length = torch.tensor(106_496, dtype=torch.long),)

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacity of 11.59 GiB of which 108.56 MiB is free. Including non-PyTorch memory, this process has 10.66 GiB memory in use. Of the allocated memory 10.11 GiB is allocated by PyTorch, and 368.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [36]:
import torch

whisper_input_features_example = [torch.randn(1, 80, 3000)]
latent_inputs_example = torch.randn(8, 8, 32, 32)

frame_idx_example = torch.tensor(0, dtype=torch.long)
librosa_length_example = torch.tensor(106_496, dtype=torch.long)

torch.onnx.export(
    model,
    (
        whisper_input_features_example,
        latent_inputs_example,
        frame_idx_example,
        librosa_length_example,
    ),
    "musetalk.onnx",
    input_names=[
        "whisper_input_features_0",
        "latent_inputs",
        "frame_idx",
        "librosa_length",
    ],
    output_names=["alias"],
    dynamo=True,
)


[torch.onnx] Obtain model graph for `MuseTalkGenerator([...]` with `torch.export.export(..., strict=False)`...





def forward(self, arg0_1: "f32[384, 80, 3]", arg1_1: "f32[384]", arg2_1: "f32[384, 384, 3]", arg3_1: "f32[384]", arg4_1: "f32[1500, 384]", arg5_1: "f32[384, 384]", arg6_1: "f32[384, 384]", arg7_1: "f32[384]", arg8_1: "f32[384, 384]", arg9_1: "f32[384]", arg10_1: "f32[384, 384]", arg11_1: "f32[384]", arg12_1: "f32[384]", arg13_1: "f32[384]", arg14_1: "f32[1536, 384]", arg15_1: "f32[1536]", arg16_1: "f32[384, 1536]", arg17_1: "f32[384]", arg18_1: "f32[384]", arg19_1: "f32[384]", arg20_1: "f32[384, 384]", arg21_1: "f32[384, 384]", arg22_1: "f32[384]", arg23_1: "f32[384, 384]", arg24_1: "f32[384]", arg25_1: "f32[384, 384]", arg26_1: "f32[384]", arg27_1: "f32[384]", arg28_1: "f32[384]", arg29_1: "f32[1536, 384]", arg30_1: "f32[1536]", arg31_1: "f32[384, 1536]", arg32_1: "f32[384]", arg33_1: "f32[384]", arg34_1: "f32[384]", arg35_1: "f32[384, 384]", arg36_1: "f32[384, 384]", arg37_1: "f32[384]", arg38_1: "f32[384, 384]", arg39_1: "f32[384]", arg40_1: "f32[384, 384]", arg41_1: "f32[384]", 

[torch.onnx] Obtain model graph for `MuseTalkGenerator([...]` with `torch.export.export(..., strict=False)`... ❌
[torch.onnx] Obtain model graph for `MuseTalkGenerator([...]` with `torch.export.export(..., strict=True)`...
[torch.onnx] Obtain model graph for `MuseTalkGenerator([...]` with `torch.export.export(..., strict=True)`... ❌


TorchExportError: Failed to export the model with torch.export. [96mThis is step 1/3[0m of exporting the model to ONNX. Next steps:
- Modify the model code for `torch.export.export` to succeed. Refer to https://pytorch.org/docs/stable/generated/exportdb/index.html for more information.
- Debug `torch.export.export` and submit a PR to PyTorch.
- Create an issue in the PyTorch GitHub repository against the [96m*torch.export*[0m component and attach the full error stack as well as reproduction scripts.

## Exception summary

<class 'torch.fx.experimental.symbolic_shapes.GuardOnDataDependentSymNode'>: Could not guard on data-dependent expression FloorToInt(50.0*(IntTrueDiv(u1, 16000))) < 0 (unhinted: FloorToInt(50.0*(IntTrueDiv(u1, 16000))) < 0).  (Size-like symbols: none)

consider using data-dependent friendly APIs such as guard_or_false, guard_or_true and statically_known_trueCaused by: (_decomp/decompositions.py:745 in slice_forward)
For more information, run with TORCH_LOGS="dynam

In [11]:
onnx_program

ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 20},
            producer_name='pytorch',
            producer_version='2.9.0+cu130',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"whisper_input_features_0"<FLOAT,[1,80,3000]>,
                %"latent_inputs"<FLOAT,[8,8,32,32]>
            ),
            outputs=(
                %"alias"<UINT8,[8,256,256,3]>
            ),
            initializers=(
                %"whisper_encoder.embed_positions.weight"<FLOAT,[1500,384]>{TorchTensor(...)},
                %"whisper_encoder.conv1.weight"<FLOAT,[384,80,3]>{TorchTensor(...)},
                %"whisper_encoder.conv1.bias"<FLOAT,[384]>{TorchTensor(...)},
                %"whisper_encoder.conv2.weight"<FLOAT,[384,384,3]>{TorchTensor(...)},
                %"whisper_encoder.conv2.bias"<FLOAT,[384]>{TorchTensor(...)},
                %"whisper_e

In [12]:
onnx_program.save("musetalk_model.onnx")

In [15]:
import onnx

# onnx_model = onnx.load("musetalk_model.onnx")
onnx.checker.check_model("musetalk_model.onnx")

In [10]:
import torch 
torch.__version__

'2.9.0+cu130'

In [16]:
onnx_program

ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 20},
            producer_name='pytorch',
            producer_version='2.9.0+cu130',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"whisper_input_features_0"<FLOAT,[1,80,3000]>,
                %"latent_inputs"<FLOAT,[8,8,32,32]>
            ),
            outputs=(
                %"alias"<UINT8,[8,256,256,3]>
            ),
            initializers=(
                %"whisper_encoder.embed_positions.weight"<FLOAT,[1500,384]>{TorchTensor(...)},
                %"whisper_encoder.conv1.weight"<FLOAT,[384,80,3]>{TorchTensor(...)},
                %"whisper_encoder.conv1.bias"<FLOAT,[384]>{TorchTensor(...)},
                %"whisper_encoder.conv2.weight"<FLOAT,[384,384,3]>{TorchTensor(...)},
                %"whisper_encoder.conv2.bias"<FLOAT,[384]>{TorchTensor(...)},
                %"whisper_e

In [1]:
import onnxruntime as ort
import numpy as np

# Load session
session = ort.InferenceSession("musetalk_model.onnx", providers=["CUDAExecutionProvider"])

print("Inputs:")
for inp in session.get_inputs():
    print(inp.name, inp.shape, inp.type)

print("Outputs:")
for out in session.get_outputs():
    print(out.name, out.shape, out.type)

Inputs:
whisper_input_features_0 [1, 80, 3000] tensor(float)
latent_inputs [8, 8, 32, 32] tensor(float)
Outputs:
alias [8, 256, 256, 3] tensor(uint8)


2025-11-14 17:12:06.158786194 [W:onnxruntime:, transformer_memcpy.cc:111 ApplyImpl] 12 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-11-14 17:12:06.164165445 [W:onnxruntime:, session_state.cc:1316 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-11-14 17:12:06.164169425 [W:onnxruntime:, session_state.cc:1318 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [16]:
model.vae.dtype

torch.float32

In [20]:
import numpy as np
import torch

def run_torch_model(
    model: MuseTalkGenerator,
    whisper_feats: torch.Tensor,
    latent_inputs: torch.Tensor,
    frame_idx: int,
    librosa_length: int,
    device: str = "cuda" if torch.cuda.is_available() else "cpu",
):
    model = model.to(device)
    model.eval()

    with torch.no_grad():
        # MuseTalkGenerator expects a *list* of tensors
        whisper_list = [whisper_feats.to(device)]
        latent_inputs = latent_inputs.to(device)

        out_np = model(
            whisper_input_features=whisper_list,
            latent_inputs=latent_inputs,
            frame_idx=frame_idx,
            librosa_length=librosa_length,
        )  # returns np.uint8 [B, H, W, 3]

    assert isinstance(out_np, np.ndarray)
    return out_np


In [8]:
import onnxruntime as ort

def run_onnx_model(
    onnx_path: str,
    whisper_feats: torch.Tensor,
    latent_inputs: torch.Tensor,
    use_cuda: bool = True,
):
    providers = ["CUDAExecutionProvider", "CPUExecutionProvider"] if use_cuda else ["CPUExecutionProvider"]

    sess = ort.InferenceSession(onnx_path, providers=providers)

    # Names from your log:
    #   whisper_input_features_0 [1, 80, 3000]
    #   latent_inputs            [8, 8, 32, 32]
    inputs = {
        "whisper_input_features_0": whisper_feats.detach().cpu().numpy().astype(np.float32),
        "latent_inputs": latent_inputs.detach().cpu().numpy().astype(np.float32),
    }

    (out_np,) = sess.run(None, inputs)  # alias: uint8 [B, H, W, 3]
    return out_np


In [9]:
def compare_torch_onnx(
    model: MuseTalkGenerator,
    onnx_path: str,
    frame_idx: int = 0,
    librosa_length: int = 106_496,
):
    # Use exactly the same shapes as in your export
    torch.manual_seed(0)
    whisper_feats = torch.randn(1, 80, 3000)       # float32
    latent_inputs = torch.randn(8, 8, 32, 32)      # float32

    torch_out = run_torch_model(
        model=model,
        whisper_feats=whisper_feats,
        latent_inputs=latent_inputs,
        frame_idx=frame_idx,
        librosa_length=librosa_length,
    )

    onnx_out = run_onnx_model(
        onnx_path=onnx_path,
        whisper_feats=whisper_feats,
        latent_inputs=latent_inputs,
        use_cuda=True,   # or False if you want pure CPU
    )

    print("Torch output shape:", torch_out.shape, torch_out.dtype)
    print("ONNX  output shape:", onnx_out.shape, onnx_out.dtype)

    # Convert to int16 to avoid uint8 wraparound on subtraction
    diff = torch_out.astype(np.int16) - onnx_out.astype(np.int16)
    abs_diff = np.abs(diff)

    print("max |diff|:", abs_diff.max())
    print("mean |diff|:", abs_diff.mean())
    print("all equal:", np.array_equal(torch_out, onnx_out))

    return torch_out, onnx_out, abs_diff


In [10]:
compare_torch_onnx(model, "musetalk_model.onnx")

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [21]:
# Use exactly the same shapes as in your export
onnx_path = "musetalk_model.onnx"
frame_idx: int = 0
librosa_length: int = 106_496
torch.manual_seed(0)
whisper_feats = torch.randn(1, 80, 3000)       # float32
latent_inputs = torch.randn(8, 8, 32, 32)      # float32

torch_out = run_torch_model(
    model=model,
    whisper_feats=whisper_feats,
    latent_inputs=latent_inputs,
    frame_idx=frame_idx,
    librosa_length=librosa_length,
)


In [ ]:
del model
del vae
del unet
del whisper
import torch
torch.cuda.empty_cache()

In [28]:

onnx_out = run_onnx_model(
    onnx_path=onnx_path,
    whisper_feats=whisper_feats,
    latent_inputs=latent_inputs,
    use_cuda=True,   # or False if you want pure CPU
)

print("Torch output shape:", torch_out.shape, torch_out.dtype)
print("ONNX  output shape:", onnx_out.shape, onnx_out.dtype)

# Convert to int16 to avoid uint8 wraparound on subtraction
diff = torch_out.astype(np.int16) - onnx_out.astype(np.int16)
abs_diff = np.abs(diff)

print("max |diff|:", abs_diff.max())
print("mean |diff|:", abs_diff.mean())
print("all equal:", np.array_equal(torch_out, onnx_out))

torch_out, onnx_out, abs_diff

2025-11-14 17:27:51.963952150 [W:onnxruntime:, transformer_memcpy.cc:111 ApplyImpl] 12 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.


Torch output shape: (8, 256, 256, 3) uint8
ONNX  output shape: (8, 256, 256, 3) uint8
max |diff|: 255
mean |diff|: 98.86735979715984
all equal: False


(array([[[[ 50,  61,  92],
          [  5,   9,  37],
          [  0,   0,  14],
          ...,
          [175, 152, 159],
          [165, 150, 158],
          [149, 138, 150]],
 
         [[ 25,  32,  75],
          [  0,   0,  15],
          [  0,   0,   0],
          ...,
          [188, 152, 160],
          [180, 152, 162],
          [159, 140, 153]],
 
         [[  6,   5,  62],
          [  0,   0,  15],
          [  0,   0,   0],
          ...,
          [189, 154, 160],
          [178, 150, 158],
          [161, 137, 152]],
 
         ...,
 
         [[ 31,  45,  67],
          [ 21,  34,  58],
          [ 17,  38,  67],
          ...,
          [124, 124, 123],
          [106, 103, 105],
          [115, 111, 116]],
 
         [[ 44,  56,  76],
          [ 26,  35,  59],
          [ 22,  37,  62],
          ...,
          [126, 124, 128],
          [110, 105, 111],
          [118, 114, 121]],
 
         [[ 59,  68,  88],
          [ 37,  45,  69],
          [ 28,  37,  62],
   